In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import time
import collections
import re
import random
import scipy.io
import glob
from sklearn.model_selection import train_test_split
from pandas_ml import ConfusionMatrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score,     recall_score, confusion_matrix, classification_report,     accuracy_score, f1_score
from sklearn.feature_selection import SelectKBest
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from nltk import PorterStemmer



In [2]:

#setting the path for the location
import os
os.chdir('F:/DataScience/spam_ham')


# ## Vectorizer
#reading the input data from the txt spam vs ham file
# In[8]:

NUM_TRAINING_EXAMPLES = 5172
NUM_TEST_EXAMPLES = 5857

BASE_DIR = './'
SPAM_DIR = 'spam/'
HAM_DIR = 'ham/'
TEST_DIR = 'test/'

vectorizer = TfidfVectorizer(input='filename',lowercase=True, stop_words="english",
                             encoding='latin-1',min_df=8) 

spam_filenames = glob.glob( BASE_DIR + SPAM_DIR + '*.txt')
print(spam_filenames)
ham_filenames = glob.glob( BASE_DIR + HAM_DIR + '*.txt')
test_filenames = [BASE_DIR + TEST_DIR + str(x) + '.txt' for x in range(NUM_TEST_EXAMPLES)]
all_filenames = spam_filenames + ham_filenames # including test_filenames

['./spam\\0006.2003-12-18.GP.spam.txt', './spam\\0008.2003-12-18.GP.spam.txt', './spam\\0017.2003-12-18.GP.spam.txt', './spam\\0018.2003-12-18.GP.spam.txt', './spam\\0026.2003-12-18.GP.spam.txt', './spam\\0032.2003-12-19.GP.spam.txt', './spam\\0040.2003-12-19.GP.spam.txt', './spam\\0041.2003-12-19.GP.spam.txt', './spam\\0046.2003-12-20.GP.spam.txt', './spam\\0052.2003-12-20.GP.spam.txt', './spam\\0054.2003-12-21.GP.spam.txt', './spam\\0058.2003-12-21.GP.spam.txt', './spam\\0061.2003-12-21.GP.spam.txt', './spam\\0062.2003-12-22.GP.spam.txt', './spam\\0065.2003-12-23.GP.spam.txt', './spam\\0069.2003-12-23.GP.spam.txt', './spam\\0072.2003-12-24.GP.spam.txt', './spam\\0073.2003-12-24.GP.spam.txt', './spam\\0075.2003-12-24.GP.spam.txt', './spam\\0080.2003-12-25.GP.spam.txt', './spam\\0082.2003-12-25.GP.spam.txt', './spam\\0084.2003-12-27.GP.spam.txt', './spam\\0088.2003-12-27.GP.spam.txt', './spam\\0093.2003-12-28.GP.spam.txt', './spam\\0094.2003-12-28.GP.spam.txt', './spam\\0100.2003-12-28

In [3]:
#converted the data to countvectorizer and created a dictionary on the same#
train_matrix = vectorizer.fit_transform(all_filenames)
test_matrix = vectorizer.transform(test_filenames)
X = train_matrix
Y = [1]*len(spam_filenames) + [0]*len(ham_filenames)

# Save as .mat 
file_dict = {}
file_dict['training_data'] = X
file_dict['training_labels'] = Y
file_dict['test_data'] = test_matrix
scipy.io.savemat('email_data.mat', file_dict)


In [4]:
#checking the dictionary sample
file_dict['training_data'].todense()


data = scipy.io.loadmat('./email_data.mat')
train_X = data['training_data'].toarray()
train_y = data['training_labels'].reshape(X.shape[0],1)
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
test_X = data['test_data'].toarray()
dt = DecisionTreeClassifier() 
    
clf = AdaBoostClassifier(n_estimators=50, base_estimator=dt,learning_rate=1).fit(X_train, y_train)
predicted_test = clf.predict(test_X)
predicted_val = clf.predict(X_val)


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
print(predicted_test)
print(predicted_val)

[0 0 0 ... 0 0 0]
[0 0 1 ... 0 0 0]


In [6]:

data = scipy.io.loadmat('./email_data.mat')
def adaboost_submission(data):
    # combine test and training data for scaling
    
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    test_X = data['test_data'].toarray()
    dt = DecisionTreeClassifier() 
    
    clf = AdaBoostClassifier(n_estimators=50, base_estimator=dt,learning_rate=1).fit(X_train, y_train)
    predicted_test = clf.predict(test_X)
    predicted_val = clf.predict(X_val)
 

    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_ada = adaboost_submission(data)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9642512077294686
F1 score: 0.9419152276295133
Recall: 0.9523809523809523
Precision: 0.9316770186335404

 clasification report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       720
           1       0.93      0.95      0.94       315

   micro avg       0.96      0.96      0.96      1035
   macro avg       0.96      0.96      0.96      1035
weighted avg       0.96      0.96      0.96      1035


 confussion matrix:
 [[698  22]
 [ 15 300]]


In [7]:

def lda_submission(data):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = LinearDiscriminantAnalysis().fit(X_train, y_train)
    predicted_test = clf.predict(test_X)
    predicted_val = clf.predict(X_val)
 

    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_lda = lda_submission(data)


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Accuracy: 0.8830917874396135
F1 score: 0.7979966611018363
Recall: 0.7587301587301587
Precision: 0.8415492957746479

 clasification report:
               precision    recall  f1-score   support

           0       0.90      0.94      0.92       720
           1       0.84      0.76      0.80       315

   micro avg       0.88      0.88      0.88      1035
   macro avg       0.87      0.85      0.86      1035
weighted avg       0.88      0.88      0.88      1035


 confussion matrix:
 [[675  45]
 [ 76 239]]


In [8]:

def qda_submission(data):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = QuadraticDiscriminantAnalysis().fit(X_train, y_train)
    predicted_test = clf.predict(test_X)
    predicted_val = clf.predict(X_val)
 

    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_qda = qda_submission(data)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Accuracy: 0.7342995169082126
F1 score: 0.6842709529276694
Recall: 0.946031746031746
Precision: 0.5359712230215827

 clasification report:
               precision    recall  f1-score   support

           0       0.96      0.64      0.77       720
           1       0.54      0.95      0.68       315

   micro avg       0.73      0.73      0.73      1035
   macro avg       0.75      0.79      0.73      1035
weighted avg       0.83      0.73      0.74      1035


 confussion matrix:
 [[462 258]
 [ 17 298]]


In [9]:

def xgboost_submission(data):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = XGBClassifier( max_depth=5,min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8).fit(X_train, y_train)
    predicted_test = clf.predict(test_X)
    predicted_val = clf.predict(X_val)
 

    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_xgm = xgboost_submission(data)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.970048309178744
F1 score: 0.9523809523809524
Recall: 0.9841269841269841
Precision: 0.9226190476190477

 clasification report:
               precision    recall  f1-score   support

           0       0.99      0.96      0.98       720
           1       0.92      0.98      0.95       315

   micro avg       0.97      0.97      0.97      1035
   macro avg       0.96      0.97      0.97      1035
weighted avg       0.97      0.97      0.97      1035


 confussion matrix:
 [[694  26]
 [  5 310]]


In [10]:

def randomforest_submission(data):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
    
    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_rf = randomforest_submission(data)

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Accuracy: 0.9642512077294686
F1 score: 0.9419152276295133
Recall: 0.9523809523809523
Precision: 0.9316770186335404

 clasification report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       720
           1       0.93      0.95      0.94       315

   micro avg       0.96      0.96      0.96      1035
   macro avg       0.96      0.96      0.96      1035
weighted avg       0.96      0.96      0.96      1035


 confussion matrix:
 [[698  22]
 [ 15 300]]


In [11]:

def logreg_submission(data):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = LogisticRegression(C=1).fit(X_train, y_train)
    
    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_log = logreg_submission(data)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9642512077294686
F1 score: 0.9419152276295133
Recall: 0.9523809523809523
Precision: 0.9316770186335404

 clasification report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       720
           1       0.93      0.95      0.94       315

   micro avg       0.96      0.96      0.96      1035
   macro avg       0.96      0.96      0.96      1035
weighted avg       0.96      0.96      0.96      1035


 confussion matrix:
 [[698  22]
 [ 15 300]]


In [12]:

def svm_submission(data, c):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = LinearSVC(C=c).fit(X_train, y_train)
    
    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test
submit_svm = svm_submission(data, 0.1)

Accuracy: 0.9642512077294686
F1 score: 0.9419152276295133
Recall: 0.9523809523809523
Precision: 0.9316770186335404

 clasification report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       720
           1       0.93      0.95      0.94       315

   micro avg       0.96      0.96      0.96      1035
   macro avg       0.96      0.96      0.96      1035
weighted avg       0.96      0.96      0.96      1035


 confussion matrix:
 [[698  22]
 [ 15 300]]


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
from scipy import stats
submit = [submit_log[i]+submit_rf[i]+submit_xgm[i]+submit_svm[i]+submit_ada[i] for i in range(len(submit_svm))]
submit = np.asarray(submit)

submit[np.where(submit==1)] = 0
submit[np.where(submit==2)] = 0
submit[np.where(submit==3)] = submit_log[np.where(submit==3)]
submit[submit > 3] = 1




In [14]:

# ### Save as .csv


df = pd.DataFrame(submit)
df.index += 1
df['Id'] = df.index
df.columns = ['Category', 'Id']
df.to_csv("submit_new.csv",header=True,columns=['Id','Category'],index = False)